In [8]:
# Importing necessary libraries
import numpy as np
import time
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [5]:
# creating checkpoint
checkpoint = "microsoft/DialoGPT-medium"
# downloading and caching tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# downloading and caching pre-trained model
model = AutoModelForCausalLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
# Building a ChatBot class with all necessary modules

class ChatBot():
    # initialize
    def __init__(self):
        # storing the chat history for chat continuity
        self.chat_history_ids = None
        # making input ids global for using them anywhere within the object
        self.bot_input_ids = None
        # flagging to check whether to end the conversation
        self.end_chat = False
        # greeting while starting each conversation
        self.welcome()

    def welcome(self):
        print("Initializing ChatBot ...")
        time.sleep(2)
        print('Type "bye" to end chat \n')
        time.sleep(3)
        # Greeting and introducing
        greeting = np.random.choice([
            "Welcome, I am Ruhi an AI Chatbot, here for your kind service",
            "Hey, Great day! I am your virtual assistant named Ruhi",
            "Hello, it's my pleasure meeting you",
            "Hi, I am Ruhi an AI Chatbot. Let's chat!"
        ])
        print("ChatBot >>  " + greeting)

    def user_input(self):
        # receiving input from user
        text = input("User    >> ")

        # ending conversation if user wishes so
        if text.lower().strip() in ['bye']:
            # turning on flag
            self.end_chat=True

            # setting a closing comment
            print('ChatBot >>  See you soon! Bye!')
            time.sleep(1)

            print('\nQuitting ChatBot ...')
        else:
            # continue chat, preprocess input text
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            self.new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, \
                                                       return_tensors='pt')

    def bot_response(self):
        # appending the new user input tokens to the chat history as chat is already begun
        if self.chat_history_ids is not None:
            self.bot_input_ids = torch.cat([self.chat_history_ids, self.new_user_input_ids], dim=-1)
        else:
            # if first entry, initializing bot_input_ids
            self.bot_input_ids = self.new_user_input_ids

        # defining the new chat_history_ids based on the preceding chats
        # generated a response while limiting the total chat history to 1000 tokens,
        self.chat_history_ids = model.generate(self.bot_input_ids, max_length=1000, \
                                               pad_token_id=tokenizer.eos_token_id)

        # taking last ouput tokens from bot
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[-1]:][0], \
                               skip_special_tokens=True)
        # setting random responses in case, our bot fails to answer
        if response == "":
            response = self.random_response()
        # printing bot response
        print('ChatBot >>  '+ response)

    # setting random responses if in case there is no response
    def random_response(self):
        i = -1
        response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)

        # iterating over history backwards to find the last token
        while response == '':
            i = i-1
            response = tokenizer.decode(self.chat_history_ids[:, self.bot_input_ids.shape[i]:][0], \
                               skip_special_tokens=True)
        # if it is a question, answer suitably
        if response.strip() == '?':
            reply = np.random.choice(["I don't know",
                                     "I am not sure"])
        # not a question? answer suitably
        else:
            reply = np.random.choice(["Great",
                                      "Fine. What's up?",
                                      "Okay"
                                     ])
        return reply

In [7]:
# building a ChatBot object
bot = ChatBot()
# starting chat
while True:
    # receiving user input
    bot.user_input()
    # checking whether to end chat
    if bot.end_chat:
        break
    bot.bot_response()

Initializing ChatBot ...
Type "bye" or "quit" or "exit" to end chat 

ChatBot >>  Hey, Great day! I am your virtual assistant
User    >> Hi


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ChatBot >>  Hi! :D
User    >> How are you 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ChatBot >>  I'm good, how are you?
User    >> exir


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


ChatBot >>  Good, I'm glad to hear that.
User    >> exit
ChatBot >>  See you soon! Bye!

Quitting ChatBot ...
